In [ ]:
pip install -U sentence-transformers

# Load Similarity model

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.spatial.distance import cityblock

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
df = pd.read_csv('stanford_df_rectified.csv')

description = df['Paragraph']
Generated_description_1 = df['Generated_description_1']
Generated_description_2 = df['Generated_description_2']


# Apply Filter

In [ ]:
import pandas as pd

# Phrases to remove
phrases_to_remove = [
    "The image depicts ",
    "In the image, ",
    "The image features ",
    "The image shows "
]

# Function to remove the phrases
def clean_description(text):
    for phrase in phrases_to_remove:
        if text.startswith(phrase):
            return text[len(phrase):].strip()
    return text


Generated_description_filtered_1 = Generated_description_1.apply(clean_description)
Generated_description_filtered_2 = Generated_description_2.apply(clean_description)


# Calculate cosine similarity

In [ ]:
# Initialize lists to store results
rows = []

# Iterate through the rows
for x in range(len(df)):
    # Example descriptions
    reference = description[x]
    generated = Generated_description_filtered_1[x]
    generated2 = Generated_description_filtered_2[x]

    # Encode the sentences
    reference_embeddings = model.encode(reference)
    generated_embeddings = model.encode(generated)
    generated2_embeddings = model.encode(generated2)

    # Compute cosine similarities
    cosine_similarity_1 = util.pytorch_cos_sim(reference_embeddings, generated_embeddings).item()
    cosine_similarity_2 = util.pytorch_cos_sim(reference_embeddings, generated2_embeddings).item()

    rows.append([reference, generated, generated2, cosine_similarity_1, cosine_similarity_2])



# Save the results in CSV file

In [ ]:
import csv

with open('cosine_similarities.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Description", "Generated_description_1", "Generated_description_2", "Cosine_similarity_1", "Cosine_similarity_2"])
    writer.writerows(rows)

print("CSV file has been created and saved as 'cosine_similarities.csv'")

CSV file has been created and saved as 'cosine_similarities_filtered.csv'
